In [140]:
import pandas as  pd

In [141]:
bio = pd.read_csv('../data/all_womens_foil_fencer_bio_data_May_13_2021_cleaned.csv', index_col='id')
rank = pd.read_csv('../data/all_womens_foil_fencer_rankings_data_May_13_2021_cleaned.csv')
bouts = pd.read_csv('../data/all_womens_foil_bout_data_May_13_2021_cleaned.csv')
tournament = pd.read_csv('../data/all_womens_foil_tournament_data_May_13_2021_cleaned.csv')

## Setup Bio Table

In [142]:
bio.head()

,name,country_code,country,hand,age,url,date_accessed
id,,,,,,,
28761,DVORKIN Mary,ISR,ISRAEL,Left,24,https://fie.org/athletes/28761,2021-05-13 21:42:17
43803,SHI Yue,CHN,CHINA,Right,22,https://fie.org/athletes/43803,2021-05-13 21:35:13
46080,TANG Nga Hei,MAC,"MACAO, CHINA",Right,20,https://fie.org/athletes/46080,2021-05-13 21:46:21
46481,WONAME Kelsey Victoria Afriyie,GHA,GHANA,Right,17,https://fie.org/athletes/46481,2021-05-13 21:48:14
47075,BRECHERET Carolina,BRA,BRAZIL,Right,16,https://fie.org/athletes/47075,2021-05-13 21:50:07


In [143]:
bio = bio.drop(['name', 'url', 'date_accessed', 'country_code', 'age'], axis=1)
bio.rename(columns={'count': 'country'})

# Convert cols to a category
for c in bio.columns:
    bio[c] = bio[c].astype('category').cat.codes

In [144]:
bio.head()

,country,hand
id,,
28761,49,0
43803,20,1
46080,59,1
46481,38,1
47075,14,1


## Setup Bouts table

In [145]:
bouts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49131 entries, 0 to 49130
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fencer_ID        49131 non-null  int64  
 1   opp_ID           49131 non-null  int64  
 2   fencer_age       49131 non-null  int64  
 3   opp_age          49131 non-null  int64  
 4   fencer_score     49131 non-null  int64  
 5   opp_score        49131 non-null  int64  
 6   winner_ID        49131 non-null  int64  
 7   fencer_curr_pts  49131 non-null  float64
 8   opp_curr_pts     49131 non-null  float64
 9   tournament_ID    49131 non-null  object 
 10  pool_ID          49131 non-null  int64  
 11  upset            49131 non-null  bool   
 12  date             49131 non-null  object 
dtypes: bool(1), float64(2), int64(8), object(2)
memory usage: 4.5+ MB


In [146]:
bouts_df = bouts.copy()

In [147]:
# Convert winner_ID to first_wins
bouts_df['first_wins'] = bouts_df['fencer_ID'] == bouts_df['winner_ID']
bouts_df['first_wins'] = bouts_df['first_wins'].astype('category')

In [148]:
bouts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49131 entries, 0 to 49130
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   fencer_ID        49131 non-null  int64   
 1   opp_ID           49131 non-null  int64   
 2   fencer_age       49131 non-null  int64   
 3   opp_age          49131 non-null  int64   
 4   fencer_score     49131 non-null  int64   
 5   opp_score        49131 non-null  int64   
 6   winner_ID        49131 non-null  int64   
 7   fencer_curr_pts  49131 non-null  float64 
 8   opp_curr_pts     49131 non-null  float64 
 9   tournament_ID    49131 non-null  object  
 10  pool_ID          49131 non-null  int64   
 11  upset            49131 non-null  bool    
 12  date             49131 non-null  object  
 13  first_wins       49131 non-null  category
dtypes: bool(1), category(1), float64(2), int64(8), object(2)
memory usage: 4.6+ MB


In [149]:
# Trim and rename bouts columns
bouts_df = bouts_df.drop(['winner_ID', 'date', 'upset', 'pool_ID',
                          'fencer_curr_pts', 'opp_curr_pts', 'tournament_ID'], axis=1)
bouts_df.rename(columns = {'fencer_age':'bout_age_1', 'opp_age':'bout_age_2', 'fencer_score':'score_1', 'opp_score':'score_2'}, inplace = True)

In [150]:
bouts_df.head()

,fencer_ID,opp_ID,bout_age_1,bout_age_2,score_1,score_2,first_wins
0,36796,19574,23,28,5,1,True
1,36796,42147,23,19,5,2,True
2,36796,49116,23,21,5,2,True
3,36796,39631,23,18,5,4,True
4,36796,23447,23,27,4,5,False


## Join the bouts to the users bio

In [151]:
df = bouts_df.merge(bio, left_on='fencer_ID', right_on='id')
df = df.rename(columns={col:col+'_1' for col in bio.columns})

df = df.merge(bio, left_on='opp_ID', right_on='id')
df = df.rename(columns={col:col+'_2' for col in bio.columns})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49131 entries, 0 to 49130
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   fencer_ID   49131 non-null  int64   
 1   opp_ID      49131 non-null  int64   
 2   bout_age_1  49131 non-null  int64   
 3   bout_age_2  49131 non-null  int64   
 4   score_1     49131 non-null  int64   
 5   score_2     49131 non-null  int64   
 6   first_wins  49131 non-null  category
 7   country_1   49131 non-null  int8    
 8   hand_1      49131 non-null  int8    
 9   country_2   49131 non-null  int8    
 10  hand_2      49131 non-null  int8    
dtypes: category(1), int64(6), int8(4)
memory usage: 2.9 MB


In [152]:
# Drop joiner columns
df = df.drop(['fencer_ID', 'opp_ID'], axis=1)

In [153]:
df.columns

Index(['bout_age_1', 'bout_age_2', 'score_1', 'score_2', 'first_wins',
       'country_1', 'hand_1', 'country_2', 'hand_2'],
      dtype='object')

In [154]:
target = 'first_wins'

In [156]:
df.head()

,bout_age_1,bout_age_2,score_1,score_2,first_wins,country_1,hand_1,country_2,hand_2
0,23,28,5,1,True,35,1,27,0
1,23,28,5,0,True,35,1,27,0
2,27,28,5,3,True,55,1,27,0
3,29,28,5,0,True,73,1,27,0
4,24,28,5,4,True,77,0,27,0


In [159]:
!pip install shap

Defaulting to user installation because normal site-packages is not writeable


In [162]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import shap

In [194]:
train, test = train_test_split(df, test_size=.3, shuffle=True)

In [195]:
feats = [x for x in df.columns if x != target]
feats

['bout_age_1',
 'bout_age_2',
 'score_1',
 'score_2',
 'country_1',
 'hand_1',
 'country_2',
 'hand_2']

In [ ]:
X = df[feats]
y = df[target]
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=1234, shuffle=True)

In [196]:
y_train = train[target]
x_train = train[feats]
y_test = test[target]
x_test = test[feats]

In [205]:
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, Y_train)

ValueError: Unknown label type: 'unknown'

In [161]:
iables with inconsistent numbe